# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import math
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 


In [2]:
class AvitoDataset(torch.utils.data.Dataset):
    def __init__(self, file_name, target_col):
        print('Загружаем ', file_name)
        df = pd.read_csv(file_name)
#        df = df[0:1024*10] # для ускорения отладки
        print('Загружен ', file_name)
        numeric_features = [ c for i, c in enumerate(df.columns) if pd.api.types.is_numeric_dtype(df.dtypes[i]) and c != target_col ]
        self.features = df[numeric_features].fillna(0)
        self.targets = df[target_col].fillna(0)
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
#        return np.array(self.features.iloc[index]).astype(float), np.array(self.targets.iloc[index]).astype(float)
        return torch.tensor(self.features.iloc[index]).float(), torch.tensor(self.targets.iloc[index]).float()
    


In [3]:
ds = AvitoDataset('train.csv', 'deal_probability')
print(len(ds))
print(ds[0])

Загружаем  train.csv
Загружен  train.csv
1503424
(tensor([ 400.,    2., 1008.]), tensor(0.1279))


In [4]:
from sklearn.model_selection import train_test_split
ds_train, ds_test = train_test_split(ds, test_size=0.25, random_state=13)
print(len(ds_train), len(ds_test))

1127568 375856


In [5]:
print(ds_train[1])

(tensor([  0.,  16., 399.]), tensor(0.))


In [6]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=1024)
print(len(dl_train))

1102


In [7]:
def log_rmse(p, t):
    return torch.sqrt(torch.mean((torch.log(p + 1) - torch.log(t + 1)) ** 2))

In [8]:
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm

In [9]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x


In [12]:
def dataset_to_tensor(ds, part):
    return torch.stack([r[part] for r in ds_train])

def get_total_loss(net, ds, loss):
    t0 = net(dataset_to_tensor(ds, 0)).reshape(-1)
    t1 = dataset_to_tensor(ds, 1)
    return loss(t0, t1)
    
def train_and_test(ds_train, ds_test, optimizer_class, n_epochs=10, lr=0.01, report_on=-1, batch_size=1024, hidden_params=100):
    net = FeedForward(3, hidden_params)
    
    print('========================================================')
    print(' Обучаем сеть: ', net)
    
    optimizer = optimizer_class(net.parameters(), lr=0.01)
    print(' Оптимизатор: ', optimizer)

    print(' До обучения')
    print('    Train loss: ', float(get_total_loss(net, ds_train, log_rmse)))
    print('    Test loss: ', float(get_total_loss(net, ds_test, log_rmse)))
    
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)
    
    report_on = max(len(dl_train) if report_on == -1 else report_on, 1)
    
    for epoch in range(n_epochs):
        print('--------- Эпоха ', epoch, '/', n_epochs, ' ------------------')
        running_loss = 0.0
        for i, data in enumerate(tqdm(dl_train)):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = log_rmse(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if (i+1) % report_on == 0:    # печатаем каждые report_on mini-batches
                print('Выучено батчей : %5d; loss: %.3f' % (i + 1, running_loss / report_on))
                running_loss = 0.0
                
    print(' Обучение закончено!')
    print(' После обучения')
    print('    Train loss: ', float(get_total_loss(net, ds_train, log_rmse)))
    print('    Test loss: ', float(get_total_loss(net, ds_test, log_rmse)))
    
train_and_test(ds_train, ds_test, torch.optim.SGD)
train_and_test(ds_train, ds_test, torch.optim.RMSprop)
train_and_test(ds_train, ds_test, torch.optim.Adam)

 Обучаем сеть:  FeedForward(
  (bn1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)
 Оптимизатор:  SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)
 До обучения
    Train loss:  0.33923378586769104
    Test loss:  0.3392110764980316
--------- Эпоха  0 / 10  ------------------


100%|██████████████████████████████████████████████████████████████████████████████| 1102/1102 [00:13<00:00, 80.35it/s]


Выучено батчей :  1102; loss: 0.207
--------- Эпоха  1 / 10  ------------------


100%|██████████████████████████████████████████████████████████████████████████████| 1102/1102 [00:13<00:00, 81.07it/s]


Выучено батчей :  1102; loss: 0.196
--------- Эпоха  2 / 10  ------------------


...

Выучено батчей :  1102; loss: 0.194
--------- Эпоха  8 / 10  ------------------


100%|██████████████████████████████████████████████████████████████████████████████| 1102/1102 [00:14<00:00, 75.80it/s]


Выучено батчей :  1102; loss: 0.194
--------- Эпоха  9 / 10  ------------------


100%|██████████████████████████████████████████████████████████████████████████████| 1102/1102 [00:14<00:00, 77.62it/s]


Выучено батчей :  1102; loss: 0.194
 Обучение закончено!
 После обучения
    Train loss:  0.19360269606113434
    Test loss:  0.19360268115997314


## Выводы
* Все оптимизаторы использовались с параметрами по умолчанию, lr=0.01
* Модель обучалась примерно до одинаковых значений функции потерь всеми 3мя оптимизаторами. Функция потерь уменьшалась от ~0.36-0.40 до 0.94. 
* Медленнее всего сходился SGD, показал близкое к финальному значение примерно после 5й эпохи, и оно (0.195) было похуже чем на 2х других оптимизаторах (0.194)
* Быстрее всех сошелся RMSprop, после 1й эпохи. Adam почти сошелся после 1й эпохи, но немного улучшился (от 0.195 до 0.194) после второй эпохи) 
* Ощущается нехватка данных для обучения. Я не смогла скачать весь набор из-за верификации телефона, train.csv поделились одногрупники. Получилось всего 3 числовых признака. Вероятно, использование признаков user и item дало бы больше информации. 